# Figure S3: Glacier signature uncertainty (glacier melt)

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr 
import geopandas as gpd
import shapely.geometry

import os
from glob import glob

import plotly.express as px
import plotly.graph_objects as go
from   plotly.subplots import make_subplots

os.chdir('/home/rooda/Dropbox/Patagonia/')

## Data

In [ ]:
metric_titles = ['Reference magnitude', 'Peak water year', 'Peak water magnitude', 'Peak water duration', 'Inter-annual variability', 
                'Seasonal contribution', 'Seasonal variability', 'Seasonal shift', 'Long-term trend', 'Long-term change']

data = pd.read_csv("MS2 Results/feature_importance_rmse.csv", index_col = 0)
data = data[data["Variable"] == "melt_on_glacier"].drop(columns= ['Variable'], axis = 0) # only this variable
data['Best'] = data[["Outline","Climate", "Volume", "GCM", "SSP", "BCM"]].idxmax(axis=1)
data = data.pivot(columns='Metric', values='Best')

data =  data[['ref_magnitude', 'peak_water_year', 'peak_water_magnitude', 'peak_water_duration', 'interannual_var', 
              'seasonal_cont', 'seasonal_var',    'seasonal_shift',       'lt_trend',            'lt_change']]
data.columns = metric_titles

In [ ]:
# merge gpd and RF dataset
basins = gpd.read_file("zip:////home/rooda/Dropbox/Patagonia/MS2 Results/zenodo/basins_boundaries.zip").set_index("basin_id")
basins['basin_name']= basins['basin_name'].replace({'Santa Cruz': 'Santa Cruz                          '})
basins = pd.merge(basins, data, left_index=True, right_index=True)


# trick for plotting using go.Choropleth (discrete data)
basins = basins.replace({"Outline":0, "Volume":1, "Climate":2, "GCM":3, "SSP":4, "BCM":5}) 

## Plot elements

In [ ]:
# basemap for background
geo_map = gpd.read_file("/home/rooda/Dropbox/ArcGIS/Chile/south_america.shp")
geo_map = geo_map[(geo_map.CC == "CI") | (geo_map.CC == "AR")]
geo_map = geo_map.dissolve(by='REGION')
geo_map["geometry"] = geo_map.simplify(0.01)

poly_gdf = shapely.geometry.Polygon([(-76, -55.7), (-76, -40.52), (-68.05, -40.52), (-68.05, -55.7), (-76, -55.8)])
poly_gdf = gpd.GeoDataFrame([1], geometry=[poly_gdf], crs=geo_map.crs)

geo_map = geo_map.clip(poly_gdf)

In [ ]:
# hydrological zone divides
geo_lines = gpd.read_file("GIS South/Basins_Patagonia_ice_divides.shp")

lats = []
lons = []

for feature in geo_lines.geometry:
    if isinstance(feature, shapely.geometry.linestring.LineString):
        linestrings = [feature]
    elif isinstance(feature, shapely.geometry.multilinestring.MultiLineString):
        linestrings = feature.geoms
    else:
        continue
    for linestring in linestrings:
        x, y = linestring.xy
        lats = np.append(lats, y)
        lons = np.append(lons, x)
        lats = np.append(lats, None)
        lons = np.append(lons, None)
        
lat_coords = [-43.2, -45.95,  -46.4,  -47.55,  -49.2,   -50.5,   -52.0, -53.1, -54.4]
lon_coords = [-71.2, -71.7,   -74.5,  -71.7,   -72.2,   -72.3,   -72.1, -71.7, -68.9]
names      = ["PPY", "PCA", "NPI-W", "NPI-E", "SPI-N", "SPI-C", "SPI-S", "GCN", "CDI"]
names      = ['<b>'+x+'</b>' for x in names]

## Plot

In [ ]:
metrics = np.array(metric_titles).reshape(2,5)

fig = make_subplots(rows=2, cols=5, horizontal_spacing = 0.001, vertical_spacing = 0.03, subplot_titles = metric_titles,
                    specs=[[{"type": "scattergeo"}, {"type": "scattergeo"}, {"type": "scattergeo"}, {"type": "scattergeo"}, {"type": "scattergeo"}],
                           [{"type": "scattergeo"}, {"type": "scattergeo"}, {"type": "scattergeo"}, {"type": "scattergeo"}, {"type": "scattergeo"}]])

for x in range(0,5):
    for y in range(0,2):
        
        fig.add_trace(go.Choropleth(geojson = eval(geo_map['geometry'].to_json()),  locations = geo_map.index, z = geo_map['iso_num'], 
                                colorscale = ["#EAEAF2", "#EAEAF2"], showscale= False, marker_line_color ='white', marker_line_width=0.1), row=y+1, col=x+1)

        fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()),  locations = basins.index, z = basins[metrics[y,x]],
                                coloraxis="coloraxis", showscale= False, marker_line_color ='white', marker_line_width=0.1), row=y+1, col=x+1)

        ## Add basin and hydrological zone names plus the hydro zone divides
        fig.add_trace(go.Scattergeo(lon = lons, lat = lats, mode = 'lines', line = dict(width = 0.8, color = 'black'),opacity = 0.5, showlegend = False),row=y+1, col=x+1)  
        fig.add_trace(go.Scattergeo(lon = lon_coords, lat=lat_coords, mode='text', text=names, textfont=dict(size=10, color = "rgba(0,0,0,0.5)"), showlegend = False),row=y+1, col=x+1)
        fig.add_scattergeo(geojson = eval(basins['geometry'].to_json()), locations = basins.index, text = basins['basin_name'], mode = 'text', showlegend = False,
                           textfont=dict(size=9, color = "rgba(0,0,0,0.3)"),row=y+1, col=x+1)

cl = px.colors.colorbrewer.Dark2
cl = [cl[0], cl[4], cl[1], cl[5], cl[6], cl[2]]
cc_scale = ([(0, cl[0])] + [(e, cl[(i + 1) // 2]) for i, e in enumerate(np.repeat(np.linspace(0, 1, 7)[1:6], 2))] + [(1, cl[5])])
colorbar  =  {"tickvals": [0,1,2,3,4,5], "ticktext": ["Glacier inventory", "Glacier volume", "Reference climate", "GCM", "SSP", "BCM"], "ticklabelposition" : "outside", "len" : 1.0, "orientation" : "h", "y" : -0.07}
coloraxis =  {"colorscale": cc_scale, "colorbar": colorbar, "cmin" : -0.5, "cmax" : 5.5}

## Layout and geos
fig.update_geos(showframe = True, framewidth = 1, framecolor = "rgba(0,0,0,0.5)", lonaxis_range=[-75, -68], lataxis_range=[-55.8, -40.5], 
                bgcolor = "#f9f9f9", showland = False, showcoastlines = False, showlakes = False)
fig.update_layout(coloraxis = coloraxis, plot_bgcolor="rgba(213,213,213,0.7)")
fig.update_layout(autosize = False, width = 1200, height = 1150, margin = dict(l=10, r=10, b=10, t=30))

fig.write_image("/home/rooda/Dropbox/Patagonia/MS2 Results/Figure_S3_signature_maps_gm.png", scale=4)
fig.show()

## Text

In [ ]:
area = pd.read_csv("MS2 Results/zenodo/dataset_historical.csv", index_col = "basin_id").area_RGI6

In [ ]:
# future SoUs
basins_subset = basins[basins.iloc[:,4:] > 2].iloc[:,4:]
basins_subset[basins_subset > 0] = 1

"Considering the signatures calculated from the glacier melt, the future SoU were the main source in only {:.1f}% ± {:.1f}% of the total glacier area".format(
((basins_subset.transpose()*area).sum(axis = 1)*100/area.sum()).mean(),((basins_subset.transpose()*area).sum(axis = 1)*100/area.sum()).std())

In [ ]:
# ref climate
basins_subset = basins[basins.iloc[:,4:] == 2].iloc[:,4:]
basins_subset[basins_subset > 0] = 1

"The reference climate was the main SoU in {:.1f}% ± {:.1f}% of the glacier area".format(
((basins_subset.transpose()*area).sum(axis = 1)*100/area.sum()).mean(),((basins_subset.transpose()*area).sum(axis = 1)*100/area.sum()).std())